In [1]:
import pandas as pd

output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['No'].tolist()

总数量：2


[1, 2]

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

url = input_.loc[a, 'Url']

dict_param = {}
list_param = url.split('?')[1].split('&') if '?' in url else []
for param in list_param:
    dict_param[param.split('=')[0]] = param.split('=')[1]

dict_param.pop('page', '')
dict_param['sort'] = 'part-number'
dict_param['limit'] = '50'

url = url.split('?')[0] + '?'
for key, value in dict_param.items():
    url += f'{key}={value}&'

list_url, i = [], 1
while True:
    b = 0
    while True:
        b += 1
        print(b, i)
        try:
            resp = requests.get(f'''{url}page={i}''',
                                headers=get_header(),
                                proxies=get_proxy(),
                                timeout=(10, 10))

            if resp.status_code == 200:
                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))

                if html.xpath('//div[@class="counter"]'):
                    break
        except KeyboardInterrupt:
            break
        except:
            continue

    # = = = = = = = = = = = = = = =

    list_url += [f'https://ecat.spectrapremium.com{url.strip()}' for url in html.xpath('//h2[@class="tuile-produit__code-produit"]/a[@tuile-produit__code-produit__lien]/@href')]

    # = = = = = = = = = = = = = = =

    i += 1

    # = = = = = = = = = = = = = = =

    if not html.xpath('//li[@class="paginator__next"]'):
        break

resp

1 1
1 2
1 3
1 4
1 5
1 6
1 7
2 7
1 8
1 9
1 10
2 10
1 11
1 12


<Response [200]>

In [6]:
len(list_url) == 0

False

In [7]:
output = pd.DataFrame({'No': [i+1 for i in range(len(list_url))],
                       'Url': list_url})

output = output.sort_values(by=['No'],
                            ascending=[True],
                            ignore_index=True)

output.to_excel('./test_url.xlsx', index=False)

output

,No,Url
0,1,https://ecat.spectrapremium.com/en/parts/BMP02A
1,1,https://ecat.spectrapremium.com/en/parts/BMP03A
2,1,https://ecat.spectrapremium.com/en/parts/BMP05A
3,1,https://ecat.spectrapremium.com/en/parts/BMP06A
4,1,https://ecat.spectrapremium.com/en/parts/BMP08A
...,...,...
547,1,https://ecat.spectrapremium.com/en/parts/VWP56A
548,1,https://ecat.spectrapremium.com/en/parts/VWP57A
549,1,https://ecat.spectrapremium.com/en/parts/VWP59A
550,1,https://ecat.spectrapremium.com/en/parts/VWP60A


In [8]:
crawler_status = 'ok'

crawler_status

'ok'

In [9]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [10]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_url_error.xlsx', index=False)

output_error

""
